In [27]:
pip install transformers

In [28]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split

In [29]:
data = pd.read_csv('/content/drive/My Drive/MinorProject2/BERT2/English_train.csv')
# data = data[['text', 'task2']]
data.head()

,tweet_id,text,task1,task2,ID
0,1123757263427186690,"hate wen females hit ah nigga with tht bro 😂😂,...",HOF,PRFN,hasoc_2020_en_2574
1,1123733301397733380,RT @airjunebug: When you're from the Bay but y...,HOF,PRFN,hasoc_2020_en_3627
2,1123734094108659712,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NOT,NONE,hasoc_2020_en_3108
3,1126951188170199049,RT @SheLoveTimothy: He ain’t on drugs he just ...,NOT,PRFN,hasoc_2020_en_3986
4,1126863510447710208,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NOT,NONE,hasoc_2020_en_5152


In [30]:
data1=pd.read_excel("/content/drive/My Drive/MinorProject2/BERT2/english_train_new.xlsx")
# data1=data1[['text','task2']]
data1.head()

,tweet_id,text,task1,task2,ID
0,1123757263427186690,"hate wen females hit ah nigga with tht bro 😂😂,...",HOF,PRFN,hasoc_2020_en_2574
1,1123733301397733380,RT @airjunebug: When you're from the Bay but y...,HOF,PRFN,hasoc_2020_en_3627
2,1123734094108659712,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NOT,NONE,hasoc_2020_en_3108
3,1126951188170199049,RT @SheLoveTimothy: He ain’t on drugs he just ...,HOF,PRFN,hasoc_2020_en_3986
4,1126863510447710208,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NOT,NONE,hasoc_2020_en_5152


In [31]:
data['task2'].value_counts()

NONE    1954
PRFN    1375
OFFN     311
HATE     154
Name: task2, dtype: int64

In [32]:
data1['task2'].value_counts()

NONE    1852
PRFN    1377
OFFN     321
HATE     158
Name: task2, dtype: int64

In [33]:

data3 = data1[data1['task2']=='OFFN']
data3.head()

,tweet_id,text,task1,task2,ID
10,1123800523499552768,Nobody:\nAngie: U getting dick ? Cus ur ass lo...,HOF,OFFN,hasoc_2020_en_2451
12,1123589327718686720,RT @tikkkii: bitches in real life be bird as f...,HOF,OFFN,hasoc_2020_en_1111
20,1130320959146057728,RT @melissafumeros: i mean honestly i’ve seen ...,HOF,OFFN,hasoc_2020_en_884
42,1126769344149483520,RT @blktoppa: Look at that ass!! https://t.co/...,HOF,OFFN,hasoc_2020_en_1929
50,1130236603291643905,Shame on you,HOF,OFFN,hasoc_2020_en_2855


In [34]:
data4=data1[data1['task2']=='HATE']
data4

,tweet_id,text,task1,task2,ID
43,1130312562124267520,@Boogie2988 I’m assuming you are talking about...,HOF,HATE,hasoc_2020_en_3785
58,1126848394184482816,RT @realDonaldTrump: James Comey is a disgrace...,HOF,HATE,hasoc_2020_en_422
59,1123605475789033472,RT @DatNiggaBooty: Put yo face in his ass 🍑😝💦 ...,HOF,HATE,hasoc_2020_en_2563
71,1126818526558347264,RT @sohmer: @realDonaldTrump The Importer pays...,HOF,HATE,hasoc_2020_en_4070
74,1126861128091480064,RT @readkropotkin: We hate nazis right?? We ex...,HOF,HATE,hasoc_2020_en_3341
...,...,...,...,...,...
3494,1126977188660752384,RT @DVATW: British Muslims.\n*More than half o...,HOF,HATE,hasoc_2020_en_2421
3531,1126814646806011905,RT @JoeyNoCollusion: You had a coup against th...,HOF,HATE,hasoc_2020_en_3786
3537,1126819247982817281,@GOPChairwoman @realDonaldTrump Cause they nev...,HOF,HATE,hasoc_2020_en_4936
3622,1127069249477009408,RT @evewhite5500: Father of Colorado school sh...,HOF,HATE,hasoc_2020_en_1838


In [35]:
data=pd.concat([data,data3,data4])
data.shape


(4273, 5)

In [36]:
data.head()

,tweet_id,text,task1,task2,ID
0,1123757263427186690,"hate wen females hit ah nigga with tht bro 😂😂,...",HOF,PRFN,hasoc_2020_en_2574
1,1123733301397733380,RT @airjunebug: When you're from the Bay but y...,HOF,PRFN,hasoc_2020_en_3627
2,1123734094108659712,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NOT,NONE,hasoc_2020_en_3108
3,1126951188170199049,RT @SheLoveTimothy: He ain’t on drugs he just ...,NOT,PRFN,hasoc_2020_en_3986
4,1126863510447710208,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NOT,NONE,hasoc_2020_en_5152


In [37]:
data.to_csv('new_english.csv')

In [38]:
data=data[['text','task2']]
data.head()

,text,task2
0,"hate wen females hit ah nigga with tht bro 😂😂,...",PRFN
1,RT @airjunebug: When you're from the Bay but y...,PRFN
2,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NONE
3,RT @SheLoveTimothy: He ain’t on drugs he just ...,PRFN
4,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NONE


In [39]:
data = data.sample(frac = 1)
data.head() 

,text,task2
3541,BRUH this review of la llorona is hella good a...,NONE
1916,@fireanddsiah Shit hard.,PRFN
1827,RT @RawStory: Trump allies warn: ‘If you touch...,NONE
3769,do it better bitch!,OFFN
13,RT @tikkkii: bitches in real life be bird as f...,OFFN


In [40]:
data['task2'] = pd.Categorical(data['task2'])
data['task2'] = data['task2'].cat.codes
data.head()

,text,task2
3541,BRUH this review of la llorona is hella good a...,1
1916,@fireanddsiah Shit hard.,3
1827,RT @RawStory: Trump allies warn: ‘If you touch...,1
3769,do it better bitch!,2
13,RT @tikkkii: bitches in real life be bird as f...,2


In [41]:
data, data_test = train_test_split(data, test_size = 0.2,random_state=42)
data.shape,data_test.shape

((3418, 2), (855, 2))

In [42]:
model_name = 'bert-base-uncased'
max_length = 64
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [43]:
bert = transformer_model.layers[0]
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
task2 = Dense(units=len(data.task2.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='task2')(pooled_output)
outputs = {'task2': task2}
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
model.summary()


Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 64)]              0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 64, 768), (None,  109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
task2 (Dense)                (None, 4)                 3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


In [44]:
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
loss = {'task2': CategoricalCrossentropy(from_logits = True)}
metric = {'task2': CategoricalAccuracy('accuracy')}

In [45]:
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
y_issue = to_categorical(data['task2'])

In [46]:
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'task2': y_issue},
    validation_split=0.2,
    batch_size=64,
    epochs=15)

Epoch 1/15
43/43 [==============================] - 66s 2s/step - loss: 1.0019 - accuracy: 0.5977 - val_loss: 0.7917 - val_accuracy: 0.7164
Epoch 2/15
43/43 [==============================] - 64s 1s/step - loss: 0.6836 - accuracy: 0.7593 - val_loss: 0.6752 - val_accuracy: 0.7617
Epoch 3/15
43/43 [==============================] - 64s 1s/step - loss: 0.5231 - accuracy: 0.8153 - val_loss: 0.5951 - val_accuracy: 0.7953
Epoch 4/15
43/43 [==============================] - 64s 1s/step - loss: 0.3825 - accuracy: 0.8734 - val_loss: 0.5911 - val_accuracy: 0.8012
Epoch 5/15
43/43 [==============================] - 64s 1s/step - loss: 0.2431 - accuracy: 0.9279 - val_loss: 0.6108 - val_accuracy: 0.8026
Epoch 6/15
43/43 [==============================] - 64s 1s/step - loss: 0.1612 - accuracy: 0.9590 - val_loss: 0.5766 - val_accuracy: 0.8333
Epoch 7/15
43/43 [==============================] - 64s 1s/step - loss: 0.1107 - accuracy: 0.9748 - val_loss: 0.6520 - val_accuracy: 0.8173
Epoch 8/15
43/43 [==

In [47]:
test_y_issue = to_categorical(data_test['task2'])
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'task2': test_y_issue}
)

27/27 [==============================] - 7s 255ms/step - loss: 0.7923 - accuracy: 0.8257


In [48]:
test_x

{'input_ids': <tf.Tensor: shape=(855, 64), dtype=int32, numpy=
array([[  101, 19387,  1030, ...,     0,     0,     0],
       [  101,  2131,  2247, ...,     0,     0,     0],
       [  101, 19387,  1030, ...,     0,     0,     0],
       ...,
       [  101, 19387,  1030, ...,     0,     0,     0],
       [  101,  8239,  3102, ...,     0,     0,     0],
       [  101, 19387,  1030, ...,     0,     0,     0]], dtype=int32)>}

In [49]:
preds = model.predict(test_x['input_ids'])

In [50]:
preds=preds['task2'].argmax(axis=-1)

In [51]:
preds

array([1, 1, 3, 1, 2, 1, 3, 2, 2, 2, 3, 3, 3, 1, 1, 1, 2, 2, 3, 3, 1, 3,
       2, 2, 1, 3, 1, 3, 3, 3, 1, 1, 3, 2, 3, 1, 3, 2, 0, 1, 1, 0, 0, 0,
       1, 0, 2, 1, 3, 1, 1, 1, 3, 1, 3, 2, 2, 3, 0, 1, 1, 1, 1, 1, 1, 0,
       3, 2, 3, 1, 1, 1, 3, 1, 3, 1, 1, 3, 3, 3, 3, 1, 0, 0, 1, 0, 1, 1,
       3, 1, 0, 1, 2, 1, 2, 3, 3, 3, 1, 3, 3, 1, 1, 3, 1, 2, 3, 3, 2, 1,
       2, 0, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 3, 1, 2, 1, 0, 1, 2, 3, 3,
       1, 2, 3, 3, 1, 1, 1, 2, 3, 1, 1, 3, 1, 3, 2, 1, 1, 1, 3, 3, 0, 1,
       2, 1, 1, 2, 2, 1, 0, 3, 1, 1, 2, 3, 2, 1, 2, 3, 1, 2, 3, 1, 1, 2,
       1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 3, 0, 1, 2, 2, 1, 0, 1, 1, 1, 3, 1,
       2, 1, 3, 3, 1, 1, 1, 0, 2, 1, 1, 3, 1, 2, 1, 2, 2, 3, 3, 1, 1, 3,
       3, 1, 0, 1, 1, 1, 3, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 3, 1, 2, 3,
       2, 1, 1, 1, 1, 2, 1, 3, 1, 3, 2, 2, 2, 0, 0, 1, 3, 3, 1, 2, 2, 2,
       3, 0, 1, 1, 3, 1, 1, 1, 0, 3, 3, 1, 3, 3, 3, 3, 1, 0, 2, 2, 1, 1,
       1, 3, 2, 0, 1, 3, 2, 3, 1, 1, 1, 3, 1, 1, 3,

In [52]:
test_y_issue=test_y_issue.argmax(axis=-1)

In [53]:
test_y_issue

array([1, 1, 3, 1, 2, 1, 3, 2, 2, 2, 3, 3, 3, 1, 1, 1, 2, 3, 3, 3, 1, 3,
       2, 3, 1, 3, 0, 3, 3, 3, 1, 1, 3, 3, 3, 1, 3, 2, 0, 1, 1, 0, 0, 1,
       1, 0, 2, 1, 3, 1, 1, 1, 3, 1, 3, 2, 2, 3, 1, 3, 1, 1, 1, 1, 2, 1,
       1, 2, 2, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 3, 3, 1, 0, 0, 1, 0, 1, 1,
       3, 1, 1, 1, 2, 1, 2, 3, 2, 3, 1, 3, 3, 1, 1, 0, 1, 1, 3, 3, 3, 1,
       2, 0, 1, 1, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 2, 1, 0, 1, 2, 3, 3,
       3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 1, 3, 1, 3, 3, 3, 1, 1, 3, 3, 0, 1,
       2, 1, 1, 0, 2, 1, 0, 3, 1, 1, 0, 1, 1, 1, 2, 3, 1, 0, 3, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 3, 3, 1, 2, 2, 1, 0, 1, 1, 1, 3, 1,
       2, 1, 2, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 2, 2, 3, 3, 1, 1, 3,
       2, 1, 3, 1, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 2, 1, 1, 2, 3, 1, 2, 3,
       3, 1, 1, 1, 1, 2, 1, 2, 1, 3, 3, 2, 3, 1, 0, 1, 3, 1, 1, 2, 3, 2,
       3, 0, 1, 1, 3, 1, 1, 1, 0, 2, 3, 1, 3, 3, 3, 3, 1, 0, 1, 3, 0, 1,
       1, 1, 2, 0, 1, 2, 2, 3, 3, 1, 1, 3, 1, 1, 3,

In [54]:
from sklearn.metrics import classification_report
print(classification_report(test_y_issue,preds))

              precision    recall  f1-score   support

           0       0.68      0.71      0.70        70
           1       0.92      0.88      0.90       379
           2       0.66      0.84      0.74       130
           3       0.84      0.77      0.80       276

    accuracy                           0.83       855
   macro avg       0.78      0.80      0.79       855
weighted avg       0.83      0.83      0.83       855

